# Projeto 2 - Ciência dos Dados

Nome: Henrique Mualem Marti

Nome: Victor Vergara

Nome: Edgard Neto

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [2]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass.json') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [3]:
#Produto escolhido:
produto = 'coca'

#Quantidade mínima de mensagens capturadas:
n = 1000
#Quantidade mínima de mensagens para a base de treinamento:
t = 250

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [4]:
file = pd.read_excel('coca.xlsx' , sheet_name='Treinamento')
file.head()

,Treinamento,Relevância
0,@r4finh0__ a mesa de natal da ines brasil com ...,1
1,rt @dcechetto: as vezes eu não quero tomar ref...,1
2,"pai bom e o meu mano, eu cheia de ressaca cheg...",1
3,@mariajoanaff aconteceu comigo esse ano logo d...,1
4,não tomem coca cola de madrugada crianças,1


#### Legenda de classificação:
0: Muito relevante - os *twetts* que expressam opiniões claras e explícitas sobre a bebida "Coca-Cola".

1: Relevante - os *twetts* que expressam alguma opinião sobre a bebida "Coca-Cola".

2: Neutro - os *twetts* que não expressam nenhuma opinião sobre a bebida "Coca-Cola".

3: Inrrelevante - os *twetts* que não estão relacionados a bebida "Coca-Cola".


In [5]:
#Limpeza
def cleanup(text):
    
    punctuation = '[!\-.:?;/()_,""''**\n]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

#for items in file.Treinamento:
    #p=cleanup(items)
    

In [6]:
file.head()

,Treinamento,Relevância
0,@r4finh0__ a mesa de natal da ines brasil com ...,1
1,rt @dcechetto: as vezes eu não quero tomar ref...,1
2,"pai bom e o meu mano, eu cheia de ressaca cheg...",1
3,@mariajoanaff aconteceu comigo esse ano logo d...,1
4,não tomem coca cola de madrugada crianças,1


In [7]:
file.Treinamento = file.Treinamento.apply(lambda x: x.lower())
file.Treinamento = file.Treinamento.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
file.Treinamento = file.Treinamento.apply(lambda x: x.translate(str.maketrans('','',string.digits)))

In [12]:
file.head()

,Treinamento,Relevância
0,rfinh a mesa de natal da ines brasil com coca...,1
1,rt dcechetto as vezes eu não quero tomar refri...,1
2,pai bom e o meu mano eu cheia de ressaca chegu...,1
3,mariajoanaff aconteceu comigo esse ano logo de...,1
4,não tomem coca cola de madrugada crianças,1


In [9]:
# Transformando quantitativa em qualitativa
file.Relevância = file.Relevância.astype('category')
file.Relevância.head()

0    1
1    1
2    1
3    1
4    1
Name: Relevância, dtype: category
Categories (4, int64): [0, 1, 2, 3]

In [10]:
file_MR=file.loc[file.Relevância==0,:]
file_R=file.loc[file.Relevância==1,:]
file_N=file.loc[file.Relevância==2,:]
file_IR=file.loc[file.Relevância==3,:]

'''
file_MR=file_MR.loc[:,['Treinamento']]
file_R=file_R.loc[:,['Treinamento']]
file_N=file_N.loc[:,['Treinamento']]
file_IR=file_IR.loc[:,['Treinamento']]
'''

file_MR

,Treinamento,Relevância
6,to tomando uma coca cedo já no intervalo tramp...,0
24,não tem uma coca cola nessa casa isso me deixa...,0
25,hoje só quero uma coca bem geladaaaaa 🤪🤤,0
26,coca cola é um ótimo desjejum recomendo,0
30,não bebo vinho só bebo coca,0
31,rt michelhcorrea pastel combina mesmo é com um...,0
34,nada melhor que pão de queijo recheado e coca ...,0
66,cocacola bebida minha preferida\n\nque mítico ...,0
74,pior noite de sono da minha vida obrigada coca...,0
77,h da matina e eu com uma vontade enorme de beb...,0


___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
MR

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis (*feito*)
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**